In [111]:
import numpy as np
import sys

In [112]:
#Given
PTX = 22       #W,  power of the transmitter (Tx) of Voyager I
R = 22*1e9*1e3 #m,  current distance (range) of Voyager I
DRX = 70       #m,  diameter of the receiver (Rx) dish
FREQ = 8.4e9   #Hz, carrier frequency
GTXDB = 48     #dB, gain of the transmitter antenna in dB
RB = 160       #bits/second, data rate or bit rate
B = 22.5*1e3   #Hz, bandwidth

############################################################################
#natural constants
c = 299792458  #m/s, speed of light
kb = 1.381e-23 #J K^{-1}, Boltzmann constant 

############################################################################
#assumed values
ETA = 0.65     #typical efficiency of antenna,see peter berlin textbook
BER = 1e-6     #Voyager I bit error rate
EbN0DB = 10.4  #db, from assumed BER = 1e-6
T01 = 20       #K,  noise temperature of the deep space network receiver
T02 = 290      #K,  noise temperature on surface of earth
F = 6          #dB, receiver noise figure
LmarginB = 20  #dB, path loss margin <- we assume this extra loss along the path to be on the safe side
#source for the assumptions, e.g. the 20K temperature: 
#https://space.stackexchange.com/questions/24338/how-to-calculate-data-rate-of-voyager-1


In [113]:
#Converting between dB and W
def db(pw):
    return 10*np.log10(pw) 

def watts(dbs):
    return 10**(dbs/10)

############################################################################
# Defining the functions to calculate the values of the variables
def prxdb(ptxdb, gtxdb, grxdb, lpathdb):
    """
        LINK BUDGET EQUATION. gives the power we get at the receiver.
        Power received by the receiver in db, depending on:
         - PTx(dB): power of the transceiver in dB
         - GTx(dB): transceiver antenna gain in dB
         - GRx(dB): receiver antenna gain in dB
         - Lpath(dB): path loss in dB
    """
    return ptxdb + gtxdb + grxdb - lpathdb #=PRx(dB)

############################################################################
def grxw(eta, dr, wavelength):
    """
        Gain of the receiver antenna in W, depending on:
         - eta:    efficiency of the antenna
         - dr(m):  diameter of the antenna in m
         - wavelength(m): wavelength of the incoming signal in m
    """
    return eta*(np.pi*dr/wavelength)**2 #=GRx(W)

############################################################################
def lpdb(wavelength, range):
    """
        Lpathdb, the power lost along the path, depending on:
         - wavelength(m): wavelength of the signal in m
         - range(m): distance (range) of the path in m
    """
    return -10*np.log10(wavelength**2 / (16*np.pi**2 * range**2))

############################################################################
def lpdbprx(ptxdb, prxmindb, gtxdb, grxdb, lmargindb):
    """
        Path loss Lpath(db) in dB depending on,
         - PRxmin(dB): minimum power required at the receiver in dB
         - PTx(dB): power of the transceiver in dB
         - GTx(dB): transceiver antenna gain in dB
         - GRx(dB): receiver antenna gain in dB
         - Lmargin(dB): path loss in dB
    """
    return ptxdb - prxmindb + gtxdb + grxdb - lmargindb

############################################################################
def snrdb(ebn0, rb, b):
    """ Signal to noise ratio SNR in dB, depending on
         - ratio Eb/N0 in dB
         - data rate or bit rate, Rb
         - bandwidth B
    """
    return ebn0 + 10*np.log10(rb / b)

############################################################################
def prxdbmin(pndb, snreqdb):
    """ Power at receiver in dB depending on 
         - Pn(dB): noise power of receiver in dB 
         - SNRreq(dB): required signal to noise ratio in dB
    """
    return pndb + snreqdb

############################################################################
def pn(t0, f, b): #noise power
    """ Noise power in Watts, depending on
         - kb: Boltzmann constant in J/K
         - T0: noise temperature at receiver in K
         - F:  Receiver noise figure
         - B:  Bandwidth in Hz
    """
    return kb * t0 * 10**(f/10) * b

In [114]:
LAMBDA = c / FREQ    # converting frequency to wavelength
PTXDB = db(PTX)
GRXDB = db(grxw(ETA, DRX, LAMBDA))
LPATHDB = lpdb(LAMBDA, R)

print("\lamdba (m):", LAMBDA)
print("PTx(dB):    ", PTXDB)
print("GRx(dB):    ", GRXDB)
print("Lpath (dB): ", LPATHDB)

\lamdba (m): 0.03568957833333333
PTx(dB):     13.424226808222063
GRx(dB):     73.92326348327545
Lpath (dB):  317.7818225595651


In [115]:
PRXDB = prxdb(PTXDB, GTXDB, GRXDB, LPATHDB)
print("1i): Link budget? see slide 28.")
print("Link budget = Power arriving at receiver, PRx(dB):", PRXDB, "dB")
print("Link budget = Power arriving at receiver, PRx(W): ", watts(PRXDB), "W")

1i): Link budget? see slide 28.
Link budget = Power arriving at receiver, PRx(dB): -182.4343322680676 dB
Link budget = Power arriving at receiver, PRx(W):  5.709088472072208e-19 W


In [116]:
SNRrequdb = snrdb(EbN0DB, RB, B)
print("1ii) Required SNR at receiver? see slide 29.")
print("SNRreq(dB): ",SNRrequdb, "db")
print("SNRreq(W): ",watts(SNRrequdb), "W")

1ii) Required SNR at receiver? see slide 29.
SNRreq(dB):  -11.080625354554376 db
SNRreq(W):  0.07797178283684872 W


In [117]:
print("2) Determine the maximum range Rmax for Eb/N0 = 0. Method: see slide 30.\n")
PNW = pn(T01, F, B)
PNDB = db(PNW)
SNRdb = snrdb(0, RB, B)

print("CASE 1: assumed temperature of receiver T0=%g K" %T01)
print("Noise power Pn(dB):", PNDB, "dB")
print("SNR(dB):           ", SNRdb,"dB")
print("Noise power Pn(W): ", watts(PNDB), "W")
print("SNR(W):            ", watts(SNRdb), "W\n")
print("From PN(dB) and SNR(dB), get min required power at receiver, PRxmin(dB).")

PRXDBMIN = prxdbmin(PNDB, SNRdb)
print("PRxmin(dB) = Pn(dB) + SNR(dB) =",PRXDBMIN, "dB")
print("PRxmin(W) = Pn(W) * SNR(W) =   ",watts(PRXDBMIN), "W\n")
print("From PRxmin(dB), get max allowable path loss, Lpathmax(dB).")

LPDBRX = lpdbprx(PTXDB, PRXDBMIN, GTXDB, GRXDB, LmarginB)
43.424, -166.1, 48, 73.85, 20
#LPDBRX = 332.85
print("Lpathmax(dB) =",LPDBRX, "dB\n")
print("From Lpathmax(dB), get max possible range, Rmax(m).")

RMAX = np.sqrt(LAMBDA**2 /(16*np.pi**2) * (10**(LPDBRX))**(1/10))
print("Rmax = %.4e m"%(RMAX))
print("     = %.4e km"%(RMAX*1e-3))

2) Determine the maximum range Rmax for Eb/N0 = 0. Method: see slide 30.

CASE 1: assumed temperature of receiver T0=20 K
Noise power Pn(dB): -166.06593807646024 dB
SNR(dB):            -21.480625354554377 dB
Noise power Pn(W):  2.4740370114047095e-17 W
SNR(W):             0.0071111111111111115 W

From PN(dB) and SNR(dB), get min required power at receiver, PRxmin(dB).
PRxmin(dB) = Pn(dB) + SNR(dB) = -187.54656343101462 dB
PRxmin(W) = Pn(W) * SNR(W) =    1.7593152081100157e-19 W

From PRxmin(dB), get max allowable path loss, Lpathmax(dB).
Lpathmax(dB) = 302.8940537225121 dB

From Lpathmax(dB), get max possible range, Rmax(m).
Rmax = 3.9631e+12 m
     = 3.9631e+09 km


In [118]:
print("2) Determine the maximum range Rmax for Eb/N0 = 0. Method: see slide 30.\n")
PNW = pn(T02, F, B)
PNDB = db(PNW)
SNRdb = snrdb(0, RB, B)

print("CASE 2: assumed temperature of receiver T0=%g K" %T02)
print("Noise power Pn(dB):", PNDB, "dB")
print("SNR(dB):           ", SNRdb, "dB\n")
print("From PN(dB) and SNR(dB), get min required power at receiver, PRxmin(dB).")

PRXDBMIN = prxdbmin(PNDB, SNRdb)
print("PRxmin(dB) = Pn(dB) + SNR(dB) =",PRXDBMIN, "dB\n")
print("From PRxmin(dB), get max allowable path loss, Lpathmax(dB).")

LPDBRX = lpdbprx(PTXDB, PRXDBMIN, GTXDB, GRXDB, LmarginB)
print("Lpathmax(dB) =",LPDBRX, "dB\n")
print("From Lpathmax(dB), get max possible range, Rmax(m).")

RMAX = np.sqrt(LAMBDA**2 /(16*np.pi**2) * (10**(LPDBRX))**(1/10))
print("Rmax = %.4e m"%(RMAX))
print("     = %.4e km"%(RMAX*1e-3))

2) Determine the maximum range Rmax for Eb/N0 = 0. Method: see slide 30.

CASE 2: assumed temperature of receiver T0=290 K
Noise power Pn(dB): -154.4522580541105 dB
SNR(dB):            -21.480625354554377 dB

From PN(dB) and SNR(dB), get min required power at receiver, PRxmin(dB).
PRxmin(dB) = Pn(dB) + SNR(dB) = -175.93288340866488 dB

From PRxmin(dB), get max allowable path loss, Lpathmax(dB).
Lpathmax(dB) = 291.2803737001624 dB

From Lpathmax(dB), get max possible range, Rmax(m).
Rmax = 1.0408e+12 m
     = 1.0408e+09 km
